# Pre-processing

In [1]:
from utils import *

In [2]:
Subjects = ['A','B']
Subject = Subjects[1]
samps = 64
Target,Target_label,NonTarget,NonTarget_label = train_data_and_label(Subject,samps)
Target, Target_label = shuffled_data(Target, Target_label)
NonTarget, NonTarget_label = shuffled_data(NonTarget, NonTarget_label)
train_P300_dataset = Target
train_P300_label = Target_label
train_non_P300_dataset = NonTarget
train_non_P300_label = NonTarget_label
print ('train_P300_dataset:' + str(train_P300_dataset.shape))
print ('train_P300_label:' + str(train_P300_label.shape))
print ('train_non_P300_dataset:'+ str(train_non_P300_dataset.shape))
print ('train_non_P300_label:' + str(train_non_P300_label.shape))

train_P300_dataset:(170, 64, 64)
train_P300_label:(170, 1)
train_non_P300_dataset:(850, 64, 64)
train_non_P300_label:(850, 1)


In [3]:
Target,Target_label,NonTarget,NonTarget_label = test_data_and_label(Subject,samps)
Target, Target_label = shuffled_data(Target, Target_label)
NonTarget, NonTarget_label = shuffled_data(NonTarget, NonTarget_label)
test_P300_dataset = Target
test_P300_label = Target_label
test_non_P300_dataset = NonTarget
test_non_P300_label = NonTarget_label
print ('test_P300_dataset:' + str(test_P300_dataset.shape))
print ('test_P300_label:' + str(test_P300_label.shape))
print ('test_non_P300_dataset:'+ str(test_non_P300_dataset.shape))
print ('test_non_P300_label:' + str(test_non_P300_label.shape))

test_P300_dataset:(200, 64, 64)
test_P300_label:(200, 1)
test_non_P300_dataset:(1000, 64, 64)
test_non_P300_label:(1000, 1)


# Experiment

In [4]:
import sklearn.metrics as skm
from sklearn import preprocessing
from torch.utils.data import Dataset,DataLoader
import torch
import torch.optim as optim
from torch.autograd import Variable
import matplotlib.pyplot as plt

In [5]:
def data_prepocessing(data):
    pd_data = np.zeros(data.shape)
    for i in range(data.shape[0]):
        pd_data[i,:,:] = preprocessing.normalize(data[i,:,:], norm='l2')
    return pd_data
def add_dimension(x, y):                                                        
    x = np.reshape(x,[x.shape[0],x.shape[2],x.shape[1]]).astype('float32')
    y = np.reshape(y,[y.shape[0],1]).astype('float32')
    return x, y

In [6]:
train_data = np.vstack((train_P300_dataset,train_non_P300_dataset))
train_label = np.vstack((train_P300_label,train_non_P300_label))
train_data, train_label = shuffled_data(train_data, train_label)

test_data = np.vstack((test_P300_dataset,test_non_P300_dataset))
test_label = np.vstack((test_P300_label,test_non_P300_label))
test_data, test_label = shuffled_data(test_data, test_label)


train_data, train_label = add_dimension(train_data,train_label)
test_data, test_label = add_dimension(test_data, test_label)
print('train_data:',train_data.shape)
print('train_label:',train_label.shape)
print('test_data:',test_data.shape)
print('test_label:',test_label.shape)

train_data: (1020, 64, 64)
train_label: (1020, 1)
test_data: (1200, 64, 64)
test_label: (1200, 1)


In [7]:
class Dataset(Dataset):
    def __init__(self, Data, Label):
        self.Data = torch.from_numpy(Data)           
        self.Label = torch.from_numpy(Label)
    def __getitem__(self, index):
        return self.Data[index], self.Label[index]
    def __len__(self):
        return len(self.Data)

In [8]:
input_channel = 64
input_length = 64
output_channel = 1              
channel_sizes = [64,64,64,64]                  
kernel_size = 7
input = torch.randn(1, 64, 64)
net = STNN_net(input_channel, input_length,output_channel, channel_sizes,kernel_size)
net(input).shape

torch.Size([1, 1])

In [9]:
LR = 0.001
epochs = 15
b_size = 32

criterion = nn.BCELoss()
optimizer = torch.optim.Adam(net.parameters(), lr=LR)
Dataset = Dataset(train_data,train_label)
Train_loader = DataLoader(Dataset,batch_size = b_size ,shuffle = True)

In [10]:
def train(epoch,train_data,train_label):
    train_loss = 0.0
    train_loss_array = []
    for step, (inputs, labels) in enumerate(Train_loader): 
        inputs, labels = Variable(inputs), Variable(labels)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
        train_loss_array.append(loss.item())
    return train_loss_array
def test(net,epoch,vaild_data,vaild_label):
    predicted = []
    inputs = torch.from_numpy(vaild_data)
    labels = torch.from_numpy(vaild_label)
    inputs, labels = Variable(inputs), Variable(labels)
    predicted = net(inputs)
    predicted = predicted.data.cpu().numpy()
    labels = labels.data.numpy()
    accuracy_score = skm.accuracy_score(labels, np.round(predicted))
    return accuracy_score

In [11]:
for epoch in range(1, epochs+1):  
    print ("\nEpoch ", epoch)
    train_loss=train(epoch,train_data,train_label)
    accuracy_score = test(net,epoch,test_data, test_label)
    print (' accuracy_score', accuracy_score)



Epoch  1
 accuracy_score 0.8408333333333333

Epoch  2
 accuracy_score 0.8675

Epoch  3
 accuracy_score 0.8683333333333333

Epoch  4
 accuracy_score 0.9016666666666666

Epoch  5
 accuracy_score 0.9033333333333333

Epoch  6
 accuracy_score 0.905

Epoch  7
 accuracy_score 0.88

Epoch  8
 accuracy_score 0.8933333333333333

Epoch  9
 accuracy_score 0.875

Epoch  10
 accuracy_score 0.9133333333333333

Epoch  11
 accuracy_score 0.91

Epoch  12
 accuracy_score 0.9158333333333334

Epoch  13
 accuracy_score 0.9133333333333333

Epoch  14
 accuracy_score 0.9108333333333334

Epoch  15
 accuracy_score 0.9116666666666666
